In [31]:
from nltk.tokenize import TreebankWordTokenizer
from pymystem3 import Mystem
from nltk.tokenize import sent_tokenize
import copy
import pandas as pd
from collections import Counter
import nltk
import unicodedata
import re
import os
#fp = open('Fuji_ansi.txt')
#data = fp.read()


In [42]:
tokenizer = TreebankWordTokenizer()
path = os.getcwd()
print("current path:", path)
path = "C:/Users/Administrator/{}_chapt_data".format('FujiUTF.txt')
os.mkdir(path)

current path: C:\Users\Administrator


In [70]:
doc_tokens = []
for line in sent_split:    
    doc_tokens += [sorted(tokenizer.tokenize(line.lower()))]

In [50]:
#TO DO!!!
def chapter_splitter(text_path):
    doc = open(text_path, encoding='utf-8')
    doc = doc.readlines()
    chapters = []
    pattern = r"[^0-3][^0-3]+$"
    f = True
    n = 0
    for line in doc:
        if re.match(pattern, line):
            print(line)
            if f: f.close()
            n += 1
            f = open('chapter{}.txt'.format(n), 'w', encoding='utf-8')
        else:
            f.write(line + '\n')
    doc.close()
    return chapters

chapter_splitter('FujiUTF.txt')

﻿    Somebody’s shouting up at а mountain…



AttributeError: 'bool' object has no attribute 'close'

In [62]:
fp = open('FujiUTF.txt', encoding='utf-8')

def lemmatizer(text_path):
    sent_split = []
    for line in text_path:
        sent_split += sent_tokenize(line)
    sentences_copy = []
    for sentence in sent_split:
        sentences_copy += [copy.copy(sentence)]

    m = Mystem()
    lemmas =[]
    for sentence in sentences_copy:
        lemmas += [m.lemmatize(sentence)]
        lemmas = [''.join(words for words in sentence) for sentence in lemmas]
        print(round(len(lemmas)/len(sentences_copy)*100, 3))
        
    output = open('%lemmatizedUTF.txt'.format(text_path), 'w', encoding='utf-8')
    
    return output.write(''.join(lemmas))

    
corpus_ch = lemmatizer(fp)
fp.close()

0.011
0.022
0.033
0.044
0.055
0.066
0.077
0.088
0.099
0.11
0.121
0.132
0.143
0.154
0.165
0.176
0.187
0.198
0.209
0.22
0.231
0.242
0.253
0.264
0.275
0.286
0.297
0.308
0.319
0.33
0.341
0.352
0.363
0.374
0.385
0.396
0.407
0.418
0.429
0.44
0.451
0.462
0.473
0.484
0.495
0.505
0.516
0.527
0.538
0.549
0.56
0.571
0.582
0.593
0.604
0.615
0.626
0.637
0.648
0.659
0.67
0.681
0.692
0.703
0.714
0.725
0.736
0.747
0.758
0.769
0.78
0.791
0.802
0.813
0.824
0.835
0.846
0.857
0.868
0.879
0.89
0.901
0.912
0.923
0.934
0.945
0.956
0.967
0.978
0.989
1.0
1.011
1.022
1.033
1.044
1.055
1.066
1.077
1.088
1.099
1.11
1.121
1.132
1.143
1.154
1.165
1.176
1.187
1.198
1.209
1.22
1.231
1.242
1.253
1.264
1.275
1.286
1.297
1.308
1.319
1.33
1.341
1.352
1.363
1.374
1.385
1.396
1.407
1.418
1.429
1.44
1.451
1.462
1.473
1.484
1.495
1.505
1.516
1.527
1.538
1.549
1.56
1.571
1.582
1.593
1.604
1.615
1.626
1.637
1.648
1.659
1.67
1.681
1.692
1.703
1.714
1.725
1.736
1.747
1.758
1.769
1.78
1.791
1.802
1.813
1.824
1.835
1.846
1.857
1.8

14.527
14.538
14.549
14.56
14.571
14.582
14.593
14.604
14.615
14.626
14.637
14.648
14.659
14.67
14.681
14.692
14.703
14.714
14.725
14.736
14.747
14.758
14.769
14.78
14.791
14.802
14.813
14.824
14.835
14.846
14.857
14.868
14.879
14.89
14.901
14.912
14.923
14.934
14.945
14.956
14.967
14.978
14.989
15.0
15.011
15.022
15.033
15.044
15.055
15.066
15.077
15.088
15.099
15.11
15.121
15.132
15.143
15.154
15.165
15.176
15.187
15.198
15.209
15.22
15.231
15.242
15.253
15.264
15.275
15.286
15.297
15.308
15.319
15.33
15.341
15.352
15.363
15.374
15.385
15.396
15.407
15.418
15.429
15.44
15.451
15.462
15.473
15.484
15.495
15.505
15.516
15.527
15.538
15.549
15.56
15.571
15.582
15.593
15.604
15.615
15.626
15.637
15.648
15.659
15.67
15.681
15.692
15.703
15.714
15.725
15.736
15.747
15.758
15.769
15.78
15.791
15.802
15.813
15.824
15.835
15.846
15.857
15.868
15.879
15.89
15.901
15.912
15.923
15.934
15.945
15.956
15.967
15.978
15.989
16.0
16.011
16.022
16.033
16.044
16.055
16.066
16.077
16.088
16.099
16.11
16

27.604
27.615
27.626
27.637
27.648
27.659
27.67
27.681
27.692
27.703
27.714
27.725
27.736
27.747
27.758
27.769
27.78
27.791
27.802
27.813
27.824
27.835
27.846
27.857
27.868
27.879
27.89
27.901
27.912
27.923
27.934
27.945
27.956
27.967
27.978
27.989
28.0
28.011
28.022
28.033
28.044
28.055
28.066
28.077
28.088
28.099
28.11
28.121
28.132
28.143
28.154
28.165
28.176
28.187
28.198
28.209
28.22
28.231
28.242
28.253
28.264
28.275
28.286
28.297
28.308
28.319
28.33
28.341
28.352
28.363
28.374
28.385
28.396
28.407
28.418
28.429
28.44
28.451
28.462
28.473
28.484
28.495
28.505
28.516
28.527
28.538
28.549
28.56
28.571
28.582
28.593
28.604
28.615
28.626
28.637
28.648
28.659
28.67
28.681
28.692
28.703
28.714
28.725
28.736
28.747
28.758
28.769
28.78
28.791
28.802
28.813
28.824
28.835
28.846
28.857
28.868
28.879
28.89
28.901
28.912
28.923
28.934
28.945
28.956
28.967
28.978
28.989
29.0
29.011
29.022
29.033
29.044
29.055
29.066
29.077
29.088
29.099
29.11
29.121
29.132
29.143
29.154
29.165
29.176
29.187
2

40.67
40.681
40.692
40.703
40.714
40.725
40.736
40.747
40.758
40.769
40.78
40.791
40.802
40.813
40.824
40.835
40.846
40.857
40.868
40.879
40.89
40.901
40.912
40.923
40.934
40.945
40.956
40.967
40.978
40.989
41.0
41.011
41.022
41.033
41.044
41.055
41.066
41.077
41.088
41.099
41.11
41.121
41.132
41.143
41.154
41.165
41.176
41.187
41.198
41.209
41.22
41.231
41.242
41.253
41.264
41.275
41.286
41.297
41.308
41.319
41.33
41.341
41.352
41.363
41.374
41.385
41.396
41.407
41.418
41.429
41.44
41.451
41.462
41.473
41.484
41.495
41.505
41.516
41.527
41.538
41.549
41.56
41.571
41.582
41.593
41.604
41.615
41.626
41.637
41.648
41.659
41.67
41.681
41.692
41.703
41.714
41.725
41.736
41.747
41.758
41.769
41.78
41.791
41.802
41.813
41.824
41.835
41.846
41.857
41.868
41.879
41.89
41.901
41.912
41.923
41.934
41.945
41.956
41.967
41.978
41.989
42.0
42.011
42.022
42.033
42.044
42.055
42.066
42.077
42.088
42.099
42.11
42.121
42.132
42.143
42.154
42.165
42.176
42.187
42.198
42.209
42.22
42.231
42.242
42.253
42

53.747
53.758
53.769
53.78
53.791
53.802
53.813
53.824
53.835
53.846
53.857
53.868
53.879
53.89
53.901
53.912
53.923
53.934
53.945
53.956
53.967
53.978
53.989
54.0
54.011
54.022
54.033
54.044
54.055
54.066
54.077
54.088
54.099
54.11
54.121
54.132
54.143
54.154
54.165
54.176
54.187
54.198
54.209
54.22
54.231
54.242
54.253
54.264
54.275
54.286
54.297
54.308
54.319
54.33
54.341
54.352
54.363
54.374
54.385
54.396
54.407
54.418
54.429
54.44
54.451
54.462
54.473
54.484
54.495
54.505
54.516
54.527
54.538
54.549
54.56
54.571
54.582
54.593
54.604
54.615
54.626
54.637
54.648
54.659
54.67
54.681
54.692
54.703
54.714
54.725
54.736
54.747
54.758
54.769
54.78
54.791
54.802
54.813
54.824
54.835
54.846
54.857
54.868
54.879
54.89
54.901
54.912
54.923
54.934
54.945
54.956
54.967
54.978
54.989
55.0
55.011
55.022
55.033
55.044
55.055
55.066
55.077
55.088
55.099
55.11
55.121
55.132
55.143
55.154
55.165
55.176
55.187
55.198
55.209
55.22
55.231
55.242
55.253
55.264
55.275
55.286
55.297
55.308
55.319
55.33
55

66.824
66.835
66.846
66.857
66.868
66.879
66.89
66.901
66.912
66.923
66.934
66.945
66.956
66.967
66.978
66.989
67.0
67.011
67.022
67.033
67.044
67.055
67.066
67.077
67.088
67.099
67.11
67.121
67.132
67.143
67.154
67.165
67.176
67.187
67.198
67.209
67.22
67.231
67.242
67.253
67.264
67.275
67.286
67.297
67.308
67.319
67.33
67.341
67.352
67.363
67.374
67.385
67.396
67.407
67.418
67.429
67.44
67.451
67.462
67.473
67.484
67.495
67.505
67.516
67.527
67.538
67.549
67.56
67.571
67.582
67.593
67.604
67.615
67.626
67.637
67.648
67.659
67.67
67.681
67.692
67.703
67.714
67.725
67.736
67.747
67.758
67.769
67.78
67.791
67.802
67.813
67.824
67.835
67.846
67.857
67.868
67.879
67.89
67.901
67.912
67.923
67.934
67.945
67.956
67.967
67.978
67.989
68.0
68.011
68.022
68.033
68.044
68.055
68.066
68.077
68.088
68.099
68.11
68.121
68.132
68.143
68.154
68.165
68.176
68.187
68.198
68.209
68.22
68.231
68.242
68.253
68.264
68.275
68.286
68.297
68.308
68.319
68.33
68.341
68.352
68.363
68.374
68.385
68.396
68.407
6

79.89
79.901
79.912
79.923
79.934
79.945
79.956
79.967
79.978
79.989
80.0
80.011
80.022
80.033
80.044
80.055
80.066
80.077
80.088
80.099
80.11
80.121
80.132
80.143
80.154
80.165
80.176
80.187
80.198
80.209
80.22
80.231
80.242
80.253
80.264
80.275
80.286
80.297
80.308
80.319
80.33
80.341
80.352
80.363
80.374
80.385
80.396
80.407
80.418
80.429
80.44
80.451
80.462
80.473
80.484
80.495
80.505
80.516
80.527
80.538
80.549
80.56
80.571
80.582
80.593
80.604
80.615
80.626
80.637
80.648
80.659
80.67
80.681
80.692
80.703
80.714
80.725
80.736
80.747
80.758
80.769
80.78
80.791
80.802
80.813
80.824
80.835
80.846
80.857
80.868
80.879
80.89
80.901
80.912
80.923
80.934
80.945
80.956
80.967
80.978
80.989
81.0
81.011
81.022
81.033
81.044
81.055
81.066
81.077
81.088
81.099
81.11
81.121
81.132
81.143
81.154
81.165
81.176
81.187
81.198
81.209
81.22
81.231
81.242
81.253
81.264
81.275
81.286
81.297
81.308
81.319
81.33
81.341
81.352
81.363
81.374
81.385
81.396
81.407
81.418
81.429
81.44
81.451
81.462
81.473
81

92.967
92.978
92.989
93.0
93.011
93.022
93.033
93.044
93.055
93.066
93.077
93.088
93.099
93.11
93.121
93.132
93.143
93.154
93.165
93.176
93.187
93.198
93.209
93.22
93.231
93.242
93.253
93.264
93.275
93.286
93.297
93.308
93.319
93.33
93.341
93.352
93.363
93.374
93.385
93.396
93.407
93.418
93.429
93.44
93.451
93.462
93.473
93.484
93.495
93.505
93.516
93.527
93.538
93.549
93.56
93.571
93.582
93.593
93.604
93.615
93.626
93.637
93.648
93.659
93.67
93.681
93.692
93.703
93.714
93.725
93.736
93.747
93.758
93.769
93.78
93.791
93.802
93.813
93.824
93.835
93.846
93.857
93.868
93.879
93.89
93.901
93.912
93.923
93.934
93.945
93.956
93.967
93.978
93.989
94.0
94.011
94.022
94.033
94.044
94.055
94.066
94.077
94.088
94.099
94.11
94.121
94.132
94.143
94.154
94.165
94.176
94.187
94.198
94.209
94.22
94.231
94.242
94.253
94.264
94.275
94.286
94.297
94.308
94.319
94.33
94.341
94.352
94.363
94.374
94.385
94.396
94.407
94.418
94.429
94.44
94.451
94.462
94.473
94.484
94.495
94.505
94.516
94.527
94.538
94.549
9

In [66]:
fp = open('Fuji_lemmatizedUTF.txt', encoding='utf-8')
data = fp.read()
sent_split = sent_tokenize(data)
len(sent_split)

"""Splits text on sentences and saves them into a list without keeping chapters splitted. 
    Removes punctuation"""
def clean_sentences(text):
    puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';',
                 ', ', '. ', '! ', '? ', ': ', '""', '— ', '« ', '» ', ') ', '(', '; ',
            ' — ', ', — ', '; ', ' (', ' «', '-', '), ', ': «', '», ', ']',
            '[', '] ', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '–', '…'))
    stopwords = nltk.corpus.stopwords.words('russian')
    
    memo_lemmas_copy = []
    for sentence in sent_split:
        memo_lemmas_copy += [copy.copy(sentence)]
        
    memo_lemmas_copy = [(letter.replace('\n', ' ') for letter in sentence) #turn it off to split chapters
                       for sentence in memo_lemmas_copy]
    memo_lemmas_copy = [''.join(letter for letter in sentence if letter not in puncs)
                        for sentence in memo_lemmas_copy]

    df = pd.DataFrame(memo_lemmas_copy, columns=['text'])
    return df

corpus = clean_sentences(data)
corpus

,text
0,﻿ Somebody’s shouting up at а mountain ...
1,FUJIⓔ стартап глотнуть бондовский мартини кокт...
2,когда катер сливаться с белый туша чужой мегая...
3,рядом с он на палуба стоять вместительный акуш...
4,добрый день сказать федор семенович и сделат...
...,...
8551,обратно исторея феминизм
8552,обратно боевой фемософия
8553,обратно сигнализирование о добродетель практи...
8554,обратно кровоточивость сердце напоказ


In [26]:
def bag_of_words(text_path, lemmatizer): #!!!ADAPT TO LEMMATIZER FUNC AND RAW TEXT!!!
    pass
    puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';',
            ', ', '. ', '! ', '? ', ': ', '""', '— ', '« ', '» ', ') ', '(', '; ',
            ' — ', ', — ', '; ', ' (', ' «', '-', '), ', ': «', '», ', ']',
            '[', '] ', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '–', '…'))
    stopwords = nltk.corpus.stopwords.words('russian')
    stopwords += set(('это', 'сказал', 'сказала', 'очень', 'просто', 'такое', 'что-то', 'тебе',
                 'вообще','сразу','несколько','почему','именно','самое','спросил','сказать','ответила'))
    all_doc_tokens = sum(doc_tokens, [])

    word_list = [x for x in all_doc_tokens 
             if x not in puncs 
             if x not in stopwords]

    lexicon = sorted(set(word_list))
    count_words = Counter(word_list)
    print('Объем лексикона (словоформы) без стопслов:', len(lexicon))
    print('Количество токенов без стопслов и знаков препинания:', len(word_list))
    #count_words.most_common(30)

In [74]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize

tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=corpus.text).toarray()

tfidf_docs = pd.DataFrame(tfidf_docs)
tfidf_docs = tfidf_docs - tfidf_docs.mean()
#tfidf_docs.shape

In [75]:
from sklearn.decomposition import PCA

pca = PCA(n_components=16)
pca = pca.fit(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
index = ['sentence{}'.format(i) for i in range(len(corpus))]
columns = ['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=index)
#pca_topic_vectors.round(2).head(106)

In [76]:
column_nums, terms = zip(*sorted(zip(tfidf.vocabulary_.values(), tfidf.vocabulary_.keys())))
#terms

In [77]:
weights = pd.DataFrame(pca.components_, columns=terms, index=['topic{}'.format(i) for i in range(16)])
pd.options.display.max_columns = 8
#weights.head(10).round(3)

In [78]:
pd.options.display.max_columns = 12
deals = weights['душа тело женщина душа тяжесть'.split()].round(3)*100
#deals

In [79]:
#deals.T.sum()

In [80]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=16, n_iter=100)
svd_topic_vectors = svd.fit_transform(tfidf_docs.values)
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns, index=index)
#svd_topic_vectors.round(3).head(6)

In [45]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=corpus.text).toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(), counter.vocabulary_.keys())))
bow_docs.columns = terms

In [81]:
#corpus.loc[50].text

In [82]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA

ldia = LDiA(n_components=16, learning_method='batch')
ldia = ldia.fit(bow_docs)
#ldia.components_.shape

In [83]:
pd.set_option('display.width', 75)
components = pd.DataFrame(ldia.components_.T, index=terms, columns=columns)
#components.round(2).head(3)

In [84]:
#components.topic11.sort_values(ascending=False)[:60]

In [19]:
print(lemmas)

NameError: name 'lemmas' is not defined

In [25]:
l = ['a', 'b', 'c', 'd', 'e']
output = open(r'Fuji_lemmatized.txt', 'w')

output.write(''.join(l))
 #!!!Must be string, not list!!!
    

5

In [61]:
test = ['\ufeff    Somebody’s shouting up at а mountain…\n', '        Pictures of Home\n', '    там, на вершина фудзи…\n', '        рама второй\n', 'часть I. Fuji ⓔ\n', '1.1.\n', 'FUJIⓔ стартап\n', 'глотнуть бондовский мартини (коктейль «над схватка», как называть его внутренний меню яхта), федор семенович откидываться на спинка шезлонг и становиться следить за ползти по синий простор катер, на который уплывать ринат.\n']
output_one = open('test.txt', 'w', encoding='utf-8')
for line in test:
    output_one.write(line)